In [2]:
!pip install sklearn
!pip install scikit_learn
!pip install pandas

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 12.2 MB 32.9 MB/s eta 0:00:01


In [1]:
import numpy as np
import keras as K
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.datasets import mnist

# 加载数据
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 处理数据
num_pixels = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')

# 标准化数据
X_train /= 255
X_test /= 255

# 对标签进行one-hot编码
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

# 定义模型
model = Sequential()
model.add(Dense(512, input_dim=num_pixels, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# 编译模型
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 训练模型
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)

# 计算每个神经元互信息
from sklearn.feature_selection import mutual_info_classif
import pandas as pd

# 获取中间层的输出
get_layer_output = K.backend.function([model.layers[0].input],
                                  [model.layers[1].output])
layer_output = get_layer_output([X_train])[0]

# 计算互信息
mi = mutual_info_classif(layer_output, y_train[:,0])
mi_df = pd.DataFrame({'mi': mi})

# 对互信息较低的神经元进行剪枝
low_mi_indices = mi_df[mi_df['mi'] < 0.05].index
pruned_model = Sequential()
pruned_model.add(Dense(512, input_dim=num_pixels, activation='relu'))

# 删除互信息较低的神经元对应的列，并随机初始化一些新的列
old_weights = model.layers[1].get_weights()[0]
old_biases = model.layers[1].get_weights()[1]
new_weights = np.delete(old_weights, low_mi_indices, axis=1)
num_new_cols = old_weights.shape[1] - new_weights.shape[1]
new_cols = np.random.randn(new_weights.shape[0], num_new_cols) * 0.01
new_weights = np.concatenate([new_weights, new_cols], axis=1)

pruned_model.add(Dense(new_weights.shape[1], activation='relu'))
pruned_model.add(Dense(num_classes, activation='softmax'))

# 拷贝训练好的参数
for i, layer in enumerate(pruned_model.layers):
    if i == 0:
        layer.set_weights(model.layers[i].get_weights())
    elif i == 1:
        layer.set_weights([new_weights, old_biases])
    else:
        layer.set_weights(model.layers[i].get_weights())

# 重新编译模型
pruned_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 重新训练
pruned_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)

2023-03-22 17:43:35.346281: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 17:43:35.478961: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-22 17:43:36.229639: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-22 17:43:36.229727: W tensorflow/

Epoch 1/10


2023-03-22 17:43:41.269374: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-03-22 17:43:41.271837: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f2accb71ae0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-22 17:43:41.271903: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2023-03-22 17:43:41.276441: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-22 17:43:41.420602: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


300/300 [==============================] - 3s 5ms/step - loss: 0.2555 - accuracy: 0.9277 - val_loss: 0.1145 - val_accuracy: 0.9642
Epoch 2/10
300/300 [==============================] - 1s 3ms/step - loss: 0.0927 - accuracy: 0.9720 - val_loss: 0.0870 - val_accuracy: 0.9710
Epoch 3/10
300/300 [==============================] - 1s 4ms/step - loss: 0.0586 - accuracy: 0.9821 - val_loss: 0.0701 - val_accuracy: 0.9771
Epoch 4/10
300/300 [==============================] - 1s 3ms/step - loss: 0.0383 - accuracy: 0.9885 - val_loss: 0.0726 - val_accuracy: 0.9766
Epoch 5/10
300/300 [==============================] - 1s 4ms/step - loss: 0.0283 - accuracy: 0.9911 - val_loss: 0.0640 - val_accuracy: 0.9800
Epoch 6/10
300/300 [==============================] - 1s 3ms/step - loss: 0.0212 - accuracy: 0.9934 - val_loss: 0.0702 - val_accuracy: 0.9793
Epoch 7/10
300/300 [==============================] - 1s 4ms/step - loss: 0.0191 - accuracy: 0.9941 - val_loss: 0.0709 - val_accuracy: 0.9807
Epoch 8/10
300/30